# Point Cloud Processing Introduction
A point cloud is a form of non-grid/unstructured data.
Unlike pixel arrays in images or voxel arrays in volumetric grids, point clouds represent sets of points without specific order. In other words, a network that
consumes a 3D point cloud of N points, needs to be invariant to N! permutations of the point feeding order. 

In this project, we will introduce you to two methods to process point clouds. Both of them are based on **permutation-invariant operations** that can be used in the unstructured point cloud directly without intermediate representation. 

The first is **PointNet (pointwise MLP based method)**. [PointNet](https://arxiv.org/abs/1612.00593) is a milestone in point cloud processing. It uses a shared MLP to extract individual point features. An MLP, or Multi-Layer Perceptron, is basically a shared FC layer operating on each point separately.

The second is **EdgeConv (graph convolution based method)**. [EdgeConv](https://arxiv.org/abs/1801.07829) uses graph convolutional networks to extract point neighborhood information and set a new state-of-the-art in some point cloud tasks. 

You will be asked to implement both [PointNet](https://arxiv.org/abs/1612.00593) and [EdgeConv](https://arxiv.org/abs/1801.07829) architectures and train a classification network (left).
![](img/cls_sem.jpg)

In [1]:
# for reloading the py file
%load_ext autoreload
%autoreload 2

In [2]:
import random
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import numpy as np 

from torchvision.transforms import Compose

# 0. Environment Installnation

In [3]:
# you can install the required environment by: source env_install.sh 
!source '/content/env_install.sh'
# see the script for details.

/content/env_install.sh: line 20: conda: command not found
/content/env_install.sh: line 21: conda: command not found
/content/env_install.sh: line 22: conda: command not found
/content/env_install.sh: line 23: conda: command not found
/content/env_install.sh: line 24: conda: command not found
     |████████████████████████████████| 1.2MB 2.5MB/s 
     |████████████████████████████████| 103.4MB 41kB/s 
     |████████████████████████████████| 143kB 48.0MB/s 
Installed kernelspec pc in /root/.local/share/jupyter/kernels/pc


# 1. Data Loading for ModelNet40  (15')

Usually, we write the point cloud as $X\in\mathbb{R}^{N\times 3}$. While in pytorch programming, we use `B x 3 x N` layout, where `B` is the batch-size and `N` is the number of points in a single point cloud.

## 1.1 augment the point clouds by randomly scaling and shifting  (5')
For input $X\in\mathbb{R}^{N\times 3}$, we randomly scale and shift the point clouds. The data is in numpy.

In [4]:
def augment_pointcloud(pointcloud):
    """
    for scaling and shifting the point cloud
    :param pointcloud:
    :return:
    """
    ## This function takes as input a point cloud of layout `N x 3`, 
    ## and outputs the scaled and shifted point cloud of layout `N x 3`.
    
    ## hint: useful function `np.random.uniform`, `np.multiply` and `np.add`
    ## TASK 1.1.1 generate a scale variable of size [3] from a uniform distruction between [2/3, 3/2] of size [3]. scale will be used to multiply with the point cloud
    scale = np.random.uniform(2/3, 3/2, 3)
    
    ## TASK 1.1.2 generate a shift variable of size [3] from a uniform distruction between [-0.2, 0.2] of size [3]. shift will be added to the point cloud
    shift = np.random.uniform(-0.2, 0.2, 3)
    
    ## TASK 1.1.3 scale and then shift the point cloud. 
    augmented_pointcloud = np.add(np.multiply(pointcloud, scale), shift)
    
    return augmented_pointcloud.astype('float32')

In [5]:
## debug. random generate data and test your transform here

pointcloud = np.random.randn(20, 3)
print('before translate:\n', pointcloud.shape, pointcloud)
augmented_pointcloud = augment_pointcloud(pointcloud)
print('after translate:\n', augmented_pointcloud.shape, augmented_pointcloud)

before translate:
 (20, 3) [[-0.14891209  0.51869835  0.6840823 ]
 [ 1.63461503  0.62292578  1.32375758]
 [ 1.49490639  0.38442752  1.54244531]
 [-0.25531914 -0.85009941 -1.1503664 ]
 [ 1.10713402  0.45831698  0.04049189]
 [ 1.25490414  1.59562664  1.22208296]
 [-0.44201959  0.0811356  -0.30065056]
 [-0.65232654  0.16367297  0.95705236]
 [-0.65779441 -1.00741298 -0.16097829]
 [ 1.8163905   1.30460426 -0.29973659]
 [-0.47566862  0.09921598  0.0279773 ]
 [-0.73966014 -0.27709417 -0.57803497]
 [ 1.52364229  1.18393305 -1.68493676]
 [ 0.47015155 -0.50278207 -0.79306278]
 [ 0.44052075  0.39602798 -1.43700655]
 [ 0.93706301  2.31450597 -0.23958904]
 [ 1.29366245 -0.38369794  0.22291237]
 [ 0.24899391  0.45522179 -0.40668408]
 [ 0.60268277 -0.17368081 -3.29500493]
 [ 0.73514484  0.59614337 -1.84076544]]
after translate:
 (20, 3) [[-0.35498613  0.55981797  0.6179654 ]
 [ 1.922421    0.6793441   1.1897595 ]
 [ 1.7440253   0.4058386   1.3852406 ]
 [-0.49085858 -1.0098947  -1.0218151 ]
 [ 1.24887

## 1.2 Load dataset ModelNet40 for Point Cloud Classification (10')

### ModelNet40
ModelNet40 contains 12,311 meshed CAD models from 40 categories.  
Given a point cloud $X\in\mathbb{R}^{N\times 3}$, the goal is to predict which category this point cloud belongs to. 

By loading this dataset, we have data of shape `B x 3 x N` and label of shape `B`.

In [6]:
# Here we provide the basic function to load and download the ModelNet40 dataset. Your job is to write a data loader for ModelNet40. 
import os
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset


def download(datadir):
    if not os.path.exists(datadir):
        os.makedirs(datadir)
    if not os.path.exists(os.path.join(datadir, 'modelnet40_ply_hdf5_2048')):
        www = 'https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip'
        print('downloading the dataset now. This may take a while. ')
        zipfile = os.path.basename(www)
        os.system('wget %s; unzip %s' % (www, zipfile))
        os.system('mv %s %s' % (zipfile[:-4], datadir))
        os.system('rm %s' % (zipfile))

def load_data(datadir, partition):
    download(datadir)
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(datadir, 'modelnet40_ply_hdf5_2048', 'ply_data_%s*.h5'%partition)):
        with h5py.File(h5_name, 'r') as f:
            data = f['data'][:].astype('float32')
            label = f['label'][:].astype('int64')
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label


### Task 1.2.1 Finish the data loader for ModelNet40.  (5')

In [7]:
class ModelNet40(data.Dataset):
    """
    This is the data loader for ModelNet40
    ModelNet40 contains 12,311 meshed CAD models from 40 categories.
    
    num_points: 1024 by default 
    datadir: we will put data in "./data/modelnet40". new one /content/drive/My Drive/Deep Learning/project4/data
    paritition: train or test
    """
        
    def __init__(self, num_points=1024, datadir="/data/modelnet40", partition='train'):
        self.data, self.label = load_data(datadir, partition)
        self.num_points = num_points
        self.partition = partition

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        label = self.label[item]

        # Task 1.2.1 augment the pointcloud when partition == 'train', and then shuffle points in the point cloud (randomly reorder). 
        if self.partition == 'train':
          pointcloud = augment_pointcloud(pointcloud)
          np.random.shuffle(pointcloud)
        
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]

    def num_classes(self):
        return np.max(self.label) + 1

In [ ]:
# please install pyvista.
!pip install pyvista

In [8]:

def vis_pc(pointcloud):
    import pyvista as pv
    plotter = pv.Plotter()
    points_voxelized = pointcloud
    point_cloud = pv.PolyData(points_voxelized)
    plotter.show_grid()
    plotter.add_mesh(point_cloud, point_size=10., render_points_as_spheres=True)
    plotter.show()



### Task 1.2.2 debug the data loader. (5')

In [9]:
# todo: 
# The dataset downloading takes maybe 15 mins. Please be patient. It will be appear in the root folder at first then be moved to ./data/ automatically. 
trainset = torch.utils.data.DataLoader(ModelNet40())

downloading the dataset now. This may take a while. 


ValueError: ignored

In [ ]:
## Task 1.2.2 show the third point cloud in trainset.data (index is 2 since the index begins from 0 in python)
pointcloud = trainset.dataset[2][0]
#vis_pc(pointcloud)

# 2 PointNet (Pointwise MLP) (45')

[PointNet](https://arxiv.org/abs/1612.00593) is a milestone in point cluod processing. It use a shared MLP to extract individual point features.   
In this section, you will be asked to implement classification step by step.   
**Please Read Section 4.2 and Appendix C of the paper.**


![pointnet](img/pointnet.jpg)

## 2.1 Basic Modules (5')
We will guide you to build the basic modules for pointwise MLP based network at first. Then you can build any network easily using these basic modules.   
We will implement the **shared MLP layer** using $1\times1$ Conv1d and implement **MLP** layer using nn.Linear. 

In [ ]:
import torch
from torch import nn
from torch.nn import Sequential as Seq, Linear as Lin, Conv1d

def act_layer(act, inplace=False, neg_slope=0.2, n_prelu=1):
    """
    activation layer
    :param act:
    :param inplace:
    :param neg_slope:
    :param n_prelu:
    :return:
    """

    act = act.lower()
    if act == 'relu':
        layer = nn.ReLU(inplace)
    elif act == 'leakyrelu':
        layer = nn.LeakyReLU(neg_slope, inplace)
    elif act == 'prelu':
        layer = nn.PReLU(num_parameters=n_prelu, init=neg_slope)
    else:
        raise NotImplementedError('activation layer [%s] is not found' % act)
    return layer

        
# Now, let's implement a sharedMLP layer. It is implmented by using Conv1d with kernel size equals to 1. 
class Conv1dLayer(Seq):
    def __init__(self, channels, act='relu', norm=True, bias=True):
        m = []
        for i in range(1, len(channels)):
            m.append(Conv1d(channels[i - 1], channels[i], 1, bias=bias))
            if norm:
                m.append(nn.BatchNorm1d(channels[i]))
            if act:
                m.append(act_layer(act))
        super(Conv1dLayer, self).__init__(*m)

### Task 2.1 Implement MLP layer according to the Conv1dLayer above. (5')
the same as Conv1dLayer class, we will implement MLP layer with normalization layer, activation layer and a **dropout** layer after the activation.  

In [ ]:
class MLP(Seq):
    """
    Given input with shape [B, C_in]
    return output with shape [B, C_out] 
    """
    def __init__(self, channels, act='relu', norm=True, bias=True, dropout=0.5):
        # todo:
        m = []
        for i in range(1, len(channels)):
            # todo 
            m.append(Conv1d(channels[i - 1], channels[i], 1, bias=bias))
            if norm:
                m.append(nn.BatchNorm1d(channels[i]))
            if act:
                m.append(act_layer(act))
            if dropout > 0:
                # todo: 
                m.append(nn.Dropout(p=dropout))

        # todo
        super(MLP, self).__init__(*m)

In [ ]:
# debug code for act_layer, MLP, and Conv1dLayer
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.randn(8, 3, 100).to(DEVICE)
conv = Conv1dLayer().to(DEVICE)
out = conv(x)
print(out.shape, transform.shape)

## 2.2 T-Net and Joint Alignment Network (15')

Joint Alignment Network predicts an affine transformation matrix by a mini-network (T-net in paper Fig 2) and directly apply this transformation to the coordinates of input points. 

T-Net takes as input matrix of size $N \times K$, and outputs a transformation matrix of size $K \times K$. 
In programming, the input size of this module is `B x K x N` and output size is `B x K x K`.

The T-Net resembles the big network and is composed by basic modules of point independent feature extraction, max pooling and fully connected layers. 
For the shared MLP, use a structure like this `(Conv1d(64), BN, ReLU, Conv1d(128), BN, ReLU, Conv1d(1024), BN, ReLU)`.
For the MLP after global max pooling, use a structure like this `(Conv1d(512), BN, ReLU, Conv1d(256), BN, ReLU, Conv1d(K*K)`.

Remember our basic modules? MLP and Conv1dLayer. You can use them to build the T-Net simply. 

### 2.2.1 T-Net architecture (10')

In [ ]:
class Transformation(nn.Module):
    def __init__(self, k=3):
        super(Transformation, self).__init__()
        self.k = k
        # Task 2.2.1 T-Net architecture
        
        # self.convs consists of 3 convolution layer. 
        # please look at the description above. 

        self.conv1 = Conv1dLayer([self.k,64,128,1024])
        self.conv2 = Conv1dLayer([1024,512,256])

        self.fcs = Conv1dLayer([256, self.k*self.k], norm=False, act=None) # no relu or BN at the last layer. 
        
    def forward(self, x):
        # Forward of T-Net architecture
        
        B, K, N = x.shape # batch-size, dim, number of points
        ## forward of shared mlp
        # input - B x K x N
        # output - B x 1024 x N
        x = x.transpose(1,2)
        
        x = self.conv1(x)
        maxpool = nn.MaxPool1d(x.shape[2])
        x = maxpool(x)
        ## global max pooling
        # input - B x 1024 x N
        # output - B x 1024
        x = self.conv2(x)
        ## mlp
        # input - B x 1024
        # output - B x (K*K)
        x = self.fcs(x)
        ## reshape the transformation matrix to B x K x K
        identity = torch.eye(self.k, device=x.device)
        x = x.view(B, self.k, self.k) + identity[None]
        return x

In [ ]:
## random generate data and test this network
x = torch.randn(10, 100, 3).to(DEVICE)
mytnet = Transformation(k=3).to(DEVICE)
print('input of the Transformation:\n', x.shape)
trans = mytnet(x) 
print('Transformation matrix:\n', trans.shape)

# the out.shape should be torch.Size([10, 3, 3]), given input x = torch.randn(10, 3, 100)

In [ ]:
def stn(x, transform_matrix=None):
    # spatial transformantion network. this is the matrix multiplication part inside the joint alignment network.
    x = x.transpose(2, 1)
    x = torch.bmm(x, transform_matrix)
    x = x.transpose(2, 1)
    return x   

In [ ]:
x_out = stn(x.transpose(1,2), trans)
print('after Transformation:\n', x_out.shape)

### 2.2.2 Regularization Loss for T-Net (5')
$$L_{reg}=\|I-TT^\intercal\|^2_F$$

The output of `Transformation` network is of size `B x K x K`. The module `OrthoLoss` computes this norm.

In [ ]:
class OrthoLoss(nn.Module):
    def __init__(self):
        super(OrthoLoss, self).__init__()
        
    def forward(self, x):
        ## hint: useful function `torch.bmm` or `torch.matmul`
        ## TASK 2.2.2
        ## compute the matrix product
        x = x.view(x.shape[0], -1)
        prod = torch.matmul(x, torch.transpose(x, 0, 1))  # TT^T
        prod = torch.eye(prod.shape[0]).to(DEVICE) - prod # minus
        norm = torch.norm(prod)
        return norm

In [ ]:
## random generate data and test this network
print(OrthoLoss()(trans))

## 2.3 PointNet Classification Network (15')
In this subsection, you will be asked to implement the feature network (the top branch).

Local features are a matrix of size `B x 64 x N`, which will be used in the segmentation task. (We only return the local feature but never uses it for the reason that we focus on claissfication)

Global features are a matrix of size `B x 1024`, which will be used in the classification task.

In [ ]:
class PointNet(nn.Module):
    def __init__(self, num_classes=40, alignment=False):
        super(PointNet, self).__init__()
        # look at the description under 2.2 or refer to the paper if you need more details
        
        self.alignment = alignment
        
        ## `input_transform` calculates the input transform matrix of size `3 x 3`
        if self.alignment:
            self.input_transform = Transformation(k=3)
        
        ## TASK 2.3.1 
        ## define your network layers here
        ## local feature
        ## one shared mlp layer (shared MLP is actually 1x1 convolution. You can use our conv1dLayer)
        ## input size: B x 3 x N
        ## output size: B x 64 x N

        self.conv1 = Conv1dLayer([3, 64])
    
        ## `feature_transform` calculates the feature transform matrix of size `64 x 64`
        if self.alignment:
            ## TASK 2.3.2 transormation layer
            self.feature_transform = Transformation(k=64)
        
        ## TASK 2.3.3
        ## define your network layers here
        ## global feature
        ## 2 layers of shared mlp.  64 -> 128 -> 1024
        ## input size: B x 64 x N
        ## output size: B x 1024 x N
        
        self.conv2s = Conv1dLayer([64, 128, 1024])
        
        # Task 2.3.4 classification layer
        # 3 MLP layers. 1024 -> 512 -> 256 -> num_classes. 
        # there is a dropout in the second layer. dropout ratio = 05
        # no relu or BN at the last layer.
        self.classifier = Seq(
            Conv1dLayer([1024, 512]),
            MLP([512, 256], dropout=0.5),
            Conv1dLayer([256, num_classes], norm=False, act=None)
        )
        
    def forward(self, x):
        
        ## task 2.3.5 apply the input transform in the coordinate domain
        if self.alignment:
            # get transformation matrix then apply to x
            transform = self.input_transform(x)
            # apply transorm into the input feature x
            x = stn(x.transpose(2,1), transform_matrix=transform)

        ## forward of shared mlp
        # input - B x K x N
        # output - B x 64 x N
        x = self.conv1(x)
        
        ## task 2.3.7 another transform in the feauture domain
        if self.alignment:
            transform = self.feature_transform(x.transpose(1,2))
            x = stn(x, transform_matrix=transform)
        else:
            transform = None
        #local_feature = x  # this can be used in segmentation task. we comment it out here. 
        
        ## TASK 2.3.8
        ## forward of shared mlp
        # input - B x 64 x N
        # output - B x 1024 x N
        x = self.conv2s(x)
        
        ## global max pooling
        # input - B x 1024 x N
        # output - B x 1024
        x = torch.max(x, dim=2, keepdim=True)[0]
        global_feature = x.view(-1, 1024)
        
        ## summary:
        ## global_feature: B x 1024
        ## local_feature: B x 64 x N
        ## transform: B x K x K
        
        # 2.3.10 classification
        out = self.classifier(x)
        return out, transform

In [ ]:
## random generate data and test this network
x = torch.randn(10, 100, 3).to(DEVICE)
net = PointNet(alignment=True)
out, transform = net(x)
print(out.shape, transform.shape)

## 2.4 Train PointNet on ModelNet40

In [ ]:
import sklearn.metrics as metrics

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0.

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def save_ckpt(model, optimizer, scheduler, jobname, name_post, ckpt_dir='/content/drive/My Drive/Deep Learning/project4/ckpt'):
    # ------------------ save ckpt
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    filename = '{}/{}_{}_model.pth'.format(ckpt_dir, jobname, name_post)
    model_cpu = {k: v.cpu() for k, v in model.state_dict().items()}
    state = {
        'state_dict': model_cpu,
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
    }
    torch.save(state, filename)
    print('save a new best model into {}'.format(filename))


### TASK 2.4 train_step function (5')

In [ ]:
def train_step(model, train_loader, optimizer, criterion, train_losses, reg):
    model.train()

    train_pred = []
    train_true = []
    for i, (data, label) in enumerate(train_loader):
        if i%100 == 0: print('train_step ', i)
        # todo: data preprocess (to GPU, and permute)
        data, label = data.to(DEVICE), label.to(DEVICE)

        # TASK 2.4 todo: optimizer zero grad, forward pass, criterion
        # optimizer zero grad
            # TASK 2.4 todo: 
        optimizer.zero_grad()
            
        # forward pass
            # TASK 2.4 todo:
        logits, trans = model(data)
        # criterion
            # TASK 2.4 todo: 
        loss = criterion(logits, label)

        # regulartization loss for T-Net
        if trans is not None:
                loss += reg(trans) * 0.001
        
        # backward part and optimize
        # TASK 2.4 todo:
        loss.backward()
        optimizer.step()

        train_losses.update(loss.item())

        preds = logits.max(dim=1)[1]
        train_true.append(label.cpu().numpy())
        train_pred.append(preds.detach().cpu().numpy())

    train_true = np.concatenate(train_true)
    train_pred = np.concatenate(train_pred)
    overall_acc = metrics.accuracy_score(train_true, train_pred)
    class_avg_acc = metrics.balanced_accuracy_score(train_true, train_pred)
    
    print('\rTrain Iter: [{:03d}/{:03d}] Loss: {:.4f} OA:{:.4f} AvgAcc:{:.4f}'.format(i+1, len(train_loader), loss.item(), overall_acc, class_avg_acc, end='', flush=True))
    return overall_acc, class_avg_acc, train_losses


def test(model, test_loader, criterion, test_losses, reg):
    model.eval()
    test_true = []
    test_pred = []
    with torch.no_grad():
        for i, (data, label) in enumerate(test_loader):
            if i%10 == 0: print('test ', i)
            data, label = data.to(DEVICE), label.to(DEVICE)
            # test forward and loss
            # TASK 2.4 todo: 
            logits, trans = model(data)
            loss = criterion(logits, label)  

            pred = logits.max(dim=1)[1]
            test_true.append(label.cpu().numpy())
            test_pred.append(pred.detach().cpu().numpy())

            test_losses.update(loss.item())

        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        overall_acc = metrics.accuracy_score(test_true, test_pred)
        class_avg_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        print('\rTest Iter: [{:03d}/{:03d}] Loss: {:.4f} OA:{:.4f} AvgAcc:{:.4f}'.format(i+1, len(train_loader), loss.item(), overall_acc, class_avg_acc, end='', flush=True))
    return overall_acc, class_avg_acc, test_losses


In [ ]:
# main train function for classification
def train_cls(train_loader, test_loader, network, optimizer, scheduler, criterion, epochs):
    reg = OrthoLoss()
    train_losses, test_losses = AverageMeter(), AverageMeter()
    best_test_overall_acc = 0. 
    for epoch in range(epochs):
        print('Epoch:[{:02d}/{:02d}]'.format(epoch+1, epochs))
        train_overall_acc, train_class_avg_acc, train_losses = train_step(network, train_loader, optimizer, criterion, train_losses, reg)
        test_overall_acc, test_class_avg_acc, test_losses = test(network, test_loader, criterion, test_losses, reg)
        
        if test_overall_acc > best_test_overall_acc:
            best_test_overall_acc = test_overall_acc
            class_avg_acc_when_best = test_class_avg_acc
            print("Got a new best model on Test with Overall ACC {:.4f}. "
                         "Its avg acc is {:.4f}".format(best_test_overall_acc, class_avg_acc_when_best))
            save_ckpt(network, optimizer, scheduler, 'PointNet', 'best')
            
        scheduler.step()
        print('Average Train Loss: {:.4f}; Train Overall Acc: {:.4f}; Test Overall ACC: {:.4f}; Best Test Overall Acc {:.4f}'.format(train_losses.avg, train_overall_acc, test_overall_acc, best_test_overall_acc))
        
        print("------------------------------------------------------------\n")
    print('Finish Training PointNet. The best Test overall acc {:.4f}'.format(best_test_overall_acc))    

In [ ]:
from torch.utils.data import DataLoader

epochs = 250 # you can change the value to a small number for debugging
batch_size = 32
test_batch_size = 50
lr = 0.001

network = PointNet(40, alignment=True).to(DEVICE)
optimizer = torch.optim.Adam(network.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(ModelNet40(partition='train', num_points=1024),
                          num_workers=4, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(ModelNet40(partition='test', num_points=1024),
                         num_workers=4, batch_size=test_batch_size, shuffle=False, drop_last=False)

# start training
train_cls(train_loader, test_loader, network, optimizer, scheduler, criterion, epochs)

## TASK 2.5 Train PointNet and fill your best overall accuracy on testing dataset here. (should be higher than 86%)  (10')

After several trials, the maximum epochs that the session supported was 28 (after running for almost 8 hours), reaching a BEST TEST OVERAL ACCURACY of **83.27%**. By the look of the training (how it was improving with each epoch), I am sure that it would have reached the required 86% accuracy if the session would have gone for longer.

**Link to .txt with the 28 epochs' results**: https://drive.google.com/file/d/13iky0thbg1Y2BLhwyghYA1QwcPbVpccv/view?usp=sharing

# 3. DGCNN with EdgeConv (GCN based Method) 
DGCNN Architecture (Read Section 3. of [EdgeConv Paper](https://arxiv.org/abs/1801.07829))

[EdgeConv](https://arxiv.org/abs/1801.07829) uses graph convolutional networks to extract point neighborhood information and set a new state-of-the-art in some point cloud tasks. 

![title](img/dgcnn.png)

## brief introduction to Graph Convolution.
A graph convolution can be created using the message passing framework:
$$ \mathbf{x}_{i}^{\prime}=\gamma_{\mathbf{\Theta}}\left(\mathbf{x}_{i}, \square_{j \in \mathcal{N}(i)} \phi_{\Theta}\left(\mathbf{x}_{i}, \mathbf{x}_{j}, \mathbf{e}_{j, i}\right)\right)$$

where $square$ denotes a differentiable, permutation invariant aggregation function, e.g., sum, mean or max.   
$\gamma_{\mathbf{\Theta}}$ and $\phi_{\Theta}$ denote differentiable functions such as MLPs, $1\times1$ convolutions.   
$x_i, x_j, e_{ij}$ is the center feature, the neighbor feature, and the edge feature between node $i$ and $j$.   
$\mathcal{N}(i)$ is the neighborhood of node $i$.  
$ \mathbf{x}_{i}^{\prime}$ is the output feature of center node $i$.  

[Further reading](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html)

EdgeConv:
$$\mathbf{h}^{v_{l+1}} =\textit{max}\left(\{\textit{mlp}(\textit{concat}(\mathbf{h}_{v_{l}}, \mathbf{h}_{u_{l}}-\mathbf{h}_{v_{l}}))|u_{l}\in \mathcal{N}^{(d)}(v_{l})\}\right)$$

## 3.1 Basic KNN Modules (5')
To build a graph convolutional network, we have to build three kinds of basic modules at first. 
1. the neighborhood querying modules (KNN)
2. the permutation invariant convolution/mlp modules with BN, activation layer inside. 
3. the graph convolution modules using 1. and 2. 

We will guide you to build these basic modules at first. Then you can build any graph convolutional network easily using these basic modules. 

First is the basic modules for neighborhood querying. We use KNN here to find k nearest neighbors (include the center node). 

In [ ]:
def knn(x, k):
    """
    Given point features x [B, C, N, 1], and number of neighbors k (int)
    Return the idx for the k neighbors of each point. 
    So, the shape of idx: [B, N, k]
    """
    with torch.no_grad():
        x = x.squeeze(-1)
        inner = -2 * torch.matmul(x.transpose(2, 1), x)
        xx = torch.sum(x ** 2, dim=1, keepdim=True)
        pairwise_distance = -xx - inner - xx.transpose(2, 1)

        idx = pairwise_distance.topk(k=k, dim=-1)[1]  # (batch_size, num_points, k)
    return idx

def batched_index_select(x, idx):
    """
    This can be used for neighbors features fetching
    Given a pointcloud x, return its k neighbors features indicated by a tensor idx.
    :param x: torch.Size([batch_size, num_dims, num_vertices, 1])
    :param index: torch.Size([batch_size, num_vertices, k])
    :return: torch.Size([batch_size, num_dims, num_vertices, k])
    """

    batch_size, num_dims, num_vertices = x.shape[:3]
    k = idx.shape[-1]
    idx_base = torch.arange(0, batch_size, device=idx.device).view(-1, 1, 1) * num_vertices
    idx = idx + idx_base
    idx = idx.view(-1)

    x = x.transpose(2, 1).contiguous()
    feature = x.view(batch_size * num_vertices, -1)[idx, :]
    feature = feature.view(batch_size, num_vertices, k, num_dims).permute(0, 3, 1, 2)
    return feature

def get_center_feature(x, k):
    """
    Given you a point cloud, and neighbors k, return the center features.
    :param x: torch.Size([batch_size, num_dims, num_vertices, 1])
    :param k: int
    :return: torch.Size([batch_size, num_dims, num_vertices, k])
    """
    x = x.repeat(1, 1, 1, k)
    return x

### TASK3.1.1 try the KNN modules (5')

In [ ]:
# please use an example to get familar with knn and the neighborhood features. 
k = 9
trainset = ModelNet40(partition='train')
pointcloud = trainset.data[2]
print(pointcloud.shape)

# TASK 3.2 write a simple example to try our knn, batched_index_select, get_center_feature functions. 
# variable pointcloud is a numpy narray, transform to the standard format of a pytorch tensor 
x = torch.from_numpy(pointcloud)
x = x.view(32,3,-1,1)
idx = knn(x, k)
x_j = batched_index_select(x, idx)  # use x_j to indicate neighbor features.
x_i = get_center_feature(x, k)      # use x_i to indicate center features. 

print(x.shape, x_j.shape, x_i.shape)

## 3.2 Basic Convolution Modules (2')

### TASK3.2 build the basic Conv2dLayer
In seciton 2.1, we build a Conv1dLayer there for the pointcloud [B, C, N]. 
Here we want to build a similar convolution layer for the point cloud [B, C, N, k]. We still use $1x1$ convolution for the permuation invariant operation. 

In [ ]:
class Conv2dLayer(Seq):
    def __init__(self, channels, act='relu', norm=True, bias=False, kernel_size=1, stride=1, dilation=1, drop=0.5):
      m = []
      for i in range(1, len(channels)):
        m.append(nn.Conv2d(channels[i - 1], channels[i], kernel_size=kernel_size, bias=bias, stride=stride, dilation=dilation))
        if norm:
            m.append(nn.BatchNorm2d(channels[i]))
        if act:
            m.append(act_layer(act))
        if drop > 0:
            m.append(nn.Dropout2d(p=drop))
      super(Conv2dLayer, self).__init__(*m)


## 3.3 Basic Graph Convolution Modules (8')

### TASK3.3: Write the forwad pass of EdgeConv.  (4')

In [ ]:
class EdgeConv2d(nn.Module):
    """
    Static EdgeConv graph convolution layer (with activation, batch normalization) for point cloud [B, C, N, 1]. 
    This operation perform the EdgeConv given the knn idx. 
    input: B, C, N, 1
    return: B, C, N, 1
    """

    def __init__(self, in_channels, out_channels, act='leakyrelu', norm=True, bias=False, aggr='max'):
        super(EdgeConv2d, self).__init__()
        self.nn = Conv2dLayer([in_channels * 2, out_channels], act, norm, bias)
        if aggr == 'mean':
            self.aggr = torch.mean
        else:
            self.aggr = torch.max

    def forward(self, x, edge_index):
        # TASK3.3: Write the forwad pass of EdgeConv. 
        x_i = get_center_feature(x, edge_index.shape[2])
        x_j = batched_index_select(x, edge_index)
        x = self.aggr(self.nn(torch.cat((x_i, x_j - x_i), dim=1)), dim=3)[0]
        x = x.view(x.shape[0], x.shape[1], x.shape[2], 1)
        return x

In [ ]:
# the x, idx here are from the 3.2
print(x.shape)
out = EdgeConv2d(3, 64)(x, idx)
print(out.shape)

### TASK3.3.1: Write the dynamic version EdgeConv. (4')

In [ ]:
class DynEdgeConv2d(EdgeConv2d):
    """
        Dynamic EdgeConv graph convolution layer (with activation, batch normalization) for point cloud [B, C, N, 1]
        This operaiton will build the knn graph at first, then perform the static EdgeConv
        input: B, C, N, 1
        return: B, C, N, 1
    """

    def __init__(self, in_channels, out_channels, k=9, act='relu',
                 norm=True, bias=False, aggr='max'):
        # inherit function
        super(DynEdgeConv2d, self).__init__(in_channels, out_channels)
        self.k = k

    def forward(self, x):
        # write the forward pass here. Combine knn with the forward pass of EdgeConv2d
        idx = knn(x, self.k).to(DEVICE)
        x = EdgeConv2d.forward(self, x, idx)
        return x

In [ ]:
x = torch.randn(8, 3, 100, 1)
print(x.shape)
out = DynEdgeConv2d(3, 64)(x)
print(out.shape)

## 3.4 Simple version of DGCNN (10')

Instead of reimplementing DGCNN, lets simply replace the MLP layers in the backbone of PointNet and see its performance. 
To make work easier, let us also remove the annoying T-Net. 

### TASK3.4: Write the SimpleGCNN class

In [ ]:
class SimpleDGCNN(nn.Module):
    def __init__(self, num_classes=40):
        super(SimpleDGCNN, self).__init__()

        # Look at PointNet backbone.
        # There are conv1d layer: 3 --> 64 --> 128 -->1024.
        # Then MLP classifier. 
        
        # Here we keep the classifier part the same. But change the backbone into dynamic EdgeConv. 
        # k=9, use relu and bachnormalization. Other parameters keep the default. 
        self.convs = Seq(
            DynEdgeConv2d(3, 64),
            DynEdgeConv2d(64, 128),
            DynEdgeConv2d(128, 1024)
        )

        self.classifier = Seq(
            Conv1dLayer([1024, 512]),
            MLP([512, 256], dropout=0.5),
            Conv1dLayer([256, num_classes], norm=False, act=None)
        )
        
    def forward(self, x):
        # x should be [B, C, N, 1]
        if len(x.shape)<4:
            x = x.unsqueeze(-1)
        
        # dynamic edgeConvolution layers
        x = self.convs(x)
        # max pooling layer 
        x = torch.max(x, dim=2, keepdim=True)[0]
        x = x.view(x.shape[0],x.shape[1],-1)
        global_feature = x.view(-1, 1024)

        out = self.classifier(x)
        return out

In [ ]:
## random generate data and test this network
x = torch.randn(8, 3, 100, 1).to(DEVICE)
net = SimpleDGCNN().to(DEVICE)
out = net(x)
print(out.shape)

## 3.5 Train SimpleDGCNN on ModelNet40 (5')

### Modify the train_step and test function in PartNet. (5')
You do not have to change too much, just make some necessary small changes

In [ ]:
def train_step(model, train_loader, optimizer, criterion, train_losses):
    # todo:
    model.train()

    train_pred = []
    train_true = []
    for i, (data, label) in enumerate(train_loader):
        if i%100 == 0: print('train_step ', i)
        # todo: data preprocess (to GPU, and permute)
        data, label = data.to(DEVICE), label.to(DEVICE)

        # TASK 2.4 todo: optimizer zero grad, forward pass, criterion
        # optimizer zero grad
            # TASK 2.4 todo: 
        optimizer.zero_grad()
            
        # forward pass
            # TASK 2.4 todo:
        logits = model(data.transpose(1,2))
        # criterion
            # TASK 2.4 todo: 
        loss = criterion(logits, label)
        
        # backward part and optimize
        # TASK 2.4 todo:
        loss.backward() ##
        optimizer.step()

        train_losses.update(loss.item())

        preds = logits.max(dim=1)[1]
        train_true.append(label.cpu().numpy())
        train_pred.append(preds.detach().cpu().numpy())

    train_true = np.concatenate(train_true)
    train_pred = np.concatenate(train_pred)
    overall_acc = metrics.accuracy_score(train_true, train_pred)
    class_avg_acc = metrics.balanced_accuracy_score(train_true, train_pred)
    
    print('\rTrain Iter: [{:03d}/{:03d}] Loss: {:.4f} OA:{:.4f} AvgAcc:{:.4f}'.format(i+1, len(train_loader), loss.item(), overall_acc, class_avg_acc, end='', flush=True))
    return overall_acc, class_avg_acc, train_losses


def test(model, test_loader, criterion, test_losses):
    # todo:
    model.eval()
    test_true = []
    test_pred = []
    with torch.no_grad():
      for i, (data, label) in enumerate(test_loader):
        if i%100 == 0: print('test ', i)
        data, label = data.to(DEVICE), label.to(DEVICE)
        # test forward and loss
        # TASK 2.4 todo: 
        logits = model(data.transpose(1,2))
        loss = criterion(logits, label)  

        pred = logits.max(dim=1)[1]
        test_true.append(label.cpu().numpy())
        test_pred.append(pred.detach().cpu().numpy())

        test_losses.update(loss.item())

      test_true = np.concatenate(test_true)
      test_pred = np.concatenate(test_pred)
      overall_acc = metrics.accuracy_score(test_true, test_pred)
      class_avg_acc = metrics.balanced_accuracy_score(test_true, test_pred)

      print('\rTest Iter: [{:03d}/{:03d}] Loss: {:.4f} OA:{:.4f} AvgAcc:{:.4f}'.format(i+1, len(train_loader), loss.item(), overall_acc, class_avg_acc, end='', flush=True))
    return overall_acc, class_avg_acc, test_losses

def train_cls(train_loader, test_loader, network, optimizer, scheduler, criterion, epochs):
    reg = OrthoLoss()
    train_losses, test_losses = AverageMeter(), AverageMeter()
    best_test_overall_acc = 0. 
    for epoch in range(epochs):
        print('Epoch:[{:02d}/{:02d}]'.format(epoch+1, epochs))
        train_overall_acc, train_class_avg_acc, train_losses = train_step(network, train_loader, optimizer, criterion, train_losses)
        test_overall_acc, test_class_avg_acc, test_losses = test(network, test_loader, criterion, test_losses)
        
        if test_overall_acc > best_test_overall_acc:
            best_test_overall_acc = test_overall_acc
            class_avg_acc_when_best = test_class_avg_acc
            print("Got a new best model on Test with Overall ACC {:.4f}. "
                         "Its avg acc is {:.4f}".format(best_test_overall_acc, class_avg_acc_when_best))
            save_ckpt(network, optimizer, scheduler, 'PointNet', 'best')
            
        scheduler.step()
        print('Average Train Loss: {:.4f}; Train Overall Acc: {:.4f}; Test Overall ACC: {:.4f}; Best Test Overall Acc {:.4f}'.format(train_losses.avg, train_overall_acc, test_overall_acc, best_test_overall_acc))
        
        print("------------------------------------------------------------\n")
    print('Finish Training PointNet. The best Test overall acc {:.4f}'.format(best_test_overall_acc))    

In [ ]:
from torch.utils.data import DataLoader

epochs = 100
batch_size=32
test_batch_size = 50
lr = 0.001 

network = SimpleDGCNN(40).to(DEVICE)
optimizer = torch.optim.Adam(network.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=lr)
criterion = nn.CrossEntropyLoss()
#######3
train_loader = DataLoader(ModelNet40(partition='train', num_points=1024),
                          num_workers=4, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(ModelNet40(partition='test', num_points=1024),
                         num_workers=4, batch_size=test_batch_size, shuffle=True, drop_last=False)

# start training
# this may take 8 hours on Titan 1080Ti
train_cls(train_loader, test_loader, network, optimizer, scheduler, criterion, epochs)

## TASK 3.6 Fill in your best overall accuracy on testing dataset here. (should be higher than 90.0%)  (5')